In [1]:
from wandrer import StravaClient


strava_credentials = StravaClient(
    access_token='c50742e984355582328f35f35191bf849e0a4098',
    refresh_token='86b84f2a3a9d1eb288b4b12d2990b7ee1aebfcff',
    client_id='8712',
    client_secret='30509b6d35b37543cfe85bee1018db8fee15e22c'
)

strava_credentials.configure_client()
activities = strava_credentials.get_activities()

ImportError: cannot import name 'StravaClient' from 'wandrer' (unknown location)